In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
%matplotlib inline

import datetime as dt
from datetime import timedelta
from datetime import datetime

from sklearn import datasets, linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
lc = LabelEncoder()

import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold, GridSearchCV
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error,mean_absolute_percentage_error

from sklearn.pipeline import make_pipeline, Pipeline

from numpy import arange
from sklearn.linear_model import Lasso, Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.kernel_ridge import KernelRidge


from sklearn import tree
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, BaggingRegressor, RandomForestRegressor
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import plot_confusion_matrix, accuracy_score
from sklearn.metrics import roc_curve, auc, plot_roc_curve, roc_auc_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import plot_roc_curve

#import precision and recall
from sklearn.metrics import precision_score, recall_score

# Trick to widen the screen
from IPython.core.display import display, HTML

#Widens the code landscape 
display(HTML("<style>.container { width:95% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows', 50)
sns.set_style('whitegrid')

In [3]:
X_test = pd.read_csv(r"C:\Users\Danny Farone\Documents\GitHub\Data-Science-Job-Salaries\data\processed\DataAnalyst_X_Test.csv")
X_train = pd.read_csv(r"C:\Users\Danny Farone\Documents\GitHub\Data-Science-Job-Salaries\data\processed\DataAnalyst_X_Train.csv")
y_test = pd.read_csv(r"C:\Users\Danny Farone\Documents\GitHub\Data-Science-Job-Salaries\data\processed\DataAnalyst_y_Test.csv")
y_train = pd.read_csv(r"C:\Users\Danny Farone\Documents\GitHub\Data-Science-Job-Salaries\data\processed\DataAnalyst_y_Train.csv")

In [ ]:
# fit initial model on the training data
logistic = LogisticRegression(max_iter = 10000, random_state = 42)
logistic.fit(X_train, y_train)

#accuracy of the model on the testing data
log_score_test = logistic.score(X_test, y_test)
#predictions for the model on the testing data
log_pred_test = logistic.predict(X_test)
#Precision of the model on the testing data
log_precision_test = precision_score(y_test, log_pred_test)
#Recall of the model on the testing data
log_recall_test = recall_score(y_test, log_pred_test)

#Print a simple confusion matrix of the teting data results
print("Logistic Regression Model Testing Data Confusion Matrix :")
print(confusion_matrix(y_test, log_pred_test)) 
#Print the scores
print("Accuracy: ", log_score_test)
print("Sensitivity: ", log_recall_test)
print("Specificity: ", log_precision_test)

In [ ]:
#set a variable for the logistic intercept
log_intercept = logistic.intercept_ 
beta_0 = log_intercept

# extract log reg coefs
coef = logistic.coef_[0]
coef = np.array(coef)
df_coef = pd.DataFrame(coef)
df_coef = df_coef.T # transpose to match column names

# column names
names = X.columns
df_coef.columns = names
df_coef = df_coef.T # transpose for better view

# sort coefficients in ascending order
df_coef = df_coef.rename(columns = {0:'logregCV_coeff'})
df_coef = df_coef.sort_values('logregCV_coeff')
df_coef = df_coef.reset_index()
df_coef = df_coef.rename(columns = {'index':'Variable_Names', 'logreg_coeff':'logregCV_coeff'})
df_coef

In [ ]:
# display barplot of log reg coefs
plt.figure(figsize=(12, 8))
sns.barplot(x="Variable_Names",y="logregCV_coeff",data= df_coef)
plt.xticks(rotation=80)
plt.title('The Barplot of Coefficients of Logistic Regression Model')
plt.xlabel('Variable Names')
plt.ylabel('Value of Coefficient')
plt.show()

In [ ]:
#create a dataframe of the the True values, False values, True Predictions, and False Predicitons for a confusion matrix
cm = pd.DataFrame(confusion_matrix(y_test, log_pred_test), index=['One_Claim','Mutliple_Claims'], 
                  columns=['One_Claim','Mutliple_Claims'])
cm.index.name = 'True'
cm.columns.name = 'Predicted'

In [ ]:
#use seaborn to create a heatmap of the confusion matrix from above
df_cm = pd.DataFrame(cm)

fig, axes = plt.subplots(nrows=1, ncols=1, sharex=True, figsize=(5, 4))
fig.suptitle('Confusion Matrix for Classification of Multiple Claims using Logistic Regression', 
             fontsize=16, y=1.05)
ax = plt.subplot()
sns.heatmap(df_cm, annot=True, cmap="Greens", annot_kws={"size": 16}, ax=ax, fmt="g")
ax.set_xlabel('Predicted', fontsize=15)
ax.set_ylabel('True', fontsize=15)
ax.xaxis.set_ticklabels(['One_Claim', 'Mutliple_Claims'], fontsize=12) 
ax.yaxis.set_ticklabels(['One_Claim', 'Mutliple_Claims'], fontsize=12, va='center')
plt.show()

In [ ]:
# plot ROC curve
y_pred_prob = logistic.predict_proba(X_test)[:,1]

fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

plt.plot([0, 1], [0, 1],'k--')
plt.plot(fpr, tpr, label='Logistic Regression')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Decision Tree ROC Curve')
plt.show();

# calculate roc curve
fpr, tpr, thresholds = roc_curve(y_test, logistic.predict(X_test))

# roc auc score
log_roc_auc = roc_auc_score(y_test, y_pred_prob)
roc_auc_format = 'ROC AUC Score: {0:.4f}'.format(log_roc_auc)
print(roc_auc_format)

In [ ]:
df_scores = pd.DataFrame({'Measures':['Accuracy','Sensitivity','Specificity','AUC'],'Logistic':[log_score,log_recall,log_precision,log_roc_auc]})
df_scores.head()

In [ ]:
X_test = pd.read_csv(r"C:\Users\Danny Farone\Documents\GitHub\Data-Science-Job-Salaries\data\processed\DataEngineer_X_Test.csv")
X_train = pd.read_csv(r"C:\Users\Danny Farone\Documents\GitHub\Data-Science-Job-Salaries\data\processed\DataEngineer_X_Train.csv")
y_test = pd.read_csv(r"C:\Users\Danny Farone\Documents\GitHub\Data-Science-Job-Salaries\data\processed\DataEngineer_y_Test.csv")
y_train = pd.read_csv(r"C:\Users\Danny Farone\Documents\GitHub\Data-Science-Job-Salaries\data\processed\DataEngineer_y_Train.csv")

In [ ]:
# fit initial model on the training data
logistic = LogisticRegression(max_iter = 10000, random_state = 42)
logistic.fit(X_train, y_train)

#accuracy of the model on the testing data
log_score_test = logistic.score(X_test, y_test)
#predictions for the model on the testing data
log_pred_test = logistic.predict(X_test)
#Precision of the model on the testing data
log_precision_test = precision_score(y_test, log_pred_test)
#Recall of the model on the testing data
log_recall_test = recall_score(y_test, log_pred_test)

#Print a simple confusion matrix of the teting data results
print("Logistic Regression Model Testing Data Confusion Matrix :")
print(confusion_matrix(y_test, log_pred_test)) 
#Print the scores
print("Accuracy: ", log_score_test)
print("Sensitivity: ", log_recall_test)
print("Specificity: ", log_precision_test)

In [ ]:
#set a variable for the logistic intercept
log_intercept = logistic.intercept_ 
beta_0 = log_intercept

# extract log reg coefs
coef = logistic.coef_[0]
coef = np.array(coef)
df_coef = pd.DataFrame(coef)
df_coef = df_coef.T # transpose to match column names

# column names
names = X.columns
df_coef.columns = names
df_coef = df_coef.T # transpose for better view

# sort coefficients in ascending order
df_coef = df_coef.rename(columns = {0:'logregCV_coeff'})
df_coef = df_coef.sort_values('logregCV_coeff')
df_coef = df_coef.reset_index()
df_coef = df_coef.rename(columns = {'index':'Variable_Names', 'logreg_coeff':'logregCV_coeff'})
df_coef

In [ ]:
# display barplot of log reg coefs
plt.figure(figsize=(12, 8))
sns.barplot(x="Variable_Names",y="logregCV_coeff",data= df_coef)
plt.xticks(rotation=80)
plt.title('The Barplot of Coefficients of Logistic Regression Model')
plt.xlabel('Variable Names')
plt.ylabel('Value of Coefficient')
plt.show()

In [ ]:
#create a dataframe of the the True values, False values, True Predictions, and False Predicitons for a confusion matrix
cm = pd.DataFrame(confusion_matrix(y_test, log_pred_test), index=['One_Claim','Mutliple_Claims'], 
                  columns=['One_Claim','Mutliple_Claims'])
cm.index.name = 'True'
cm.columns.name = 'Predicted'

In [ ]:
#use seaborn to create a heatmap of the confusion matrix from above
df_cm = pd.DataFrame(cm)

fig, axes = plt.subplots(nrows=1, ncols=1, sharex=True, figsize=(5, 4))
fig.suptitle('Confusion Matrix for Classification of Multiple Claims using Logistic Regression', 
             fontsize=16, y=1.05)
ax = plt.subplot()
sns.heatmap(df_cm, annot=True, cmap="Greens", annot_kws={"size": 16}, ax=ax, fmt="g")
ax.set_xlabel('Predicted', fontsize=15)
ax.set_ylabel('True', fontsize=15)
ax.xaxis.set_ticklabels(['One_Claim', 'Mutliple_Claims'], fontsize=12) 
ax.yaxis.set_ticklabels(['One_Claim', 'Mutliple_Claims'], fontsize=12, va='center')
plt.show()

In [ ]:
# plot ROC curve
y_pred_prob = logistic.predict_proba(X_test)[:,1]

fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

plt.plot([0, 1], [0, 1],'k--')
plt.plot(fpr, tpr, label='Logistic Regression')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Decision Tree ROC Curve')
plt.show();

# calculate roc curve
fpr, tpr, thresholds = roc_curve(y_test, logistic.predict(X_test))

# roc auc score
log_roc_auc = roc_auc_score(y_test, y_pred_prob)
roc_auc_format = 'ROC AUC Score: {0:.4f}'.format(log_roc_auc)
print(roc_auc_format)

In [ ]:
df_scores = pd.DataFrame({'Measures':['Accuracy','Sensitivity','Specificity','AUC'],'Logistic':[log_score,log_recall,log_precision,log_roc_auc]})
df_scores.head()

In [ ]:
X_test = pd.read_csv(r"C:\Users\Danny Farone\Documents\GitHub\Data-Science-Job-Salaries\data\processed\DataScientist_X_Test.csv")
X_train = pd.read_csv(r"C:\Users\Danny Farone\Documents\GitHub\Data-Science-Job-Salaries\data\processed\DataScientist_X_Train.csv")
y_test = pd.read_csv(r"C:\Users\Danny Farone\Documents\GitHub\Data-Science-Job-Salaries\data\processed\DataScientist_y_Test.csv")
y_train = pd.read_csv(r"C:\Users\Danny Farone\Documents\GitHub\Data-Science-Job-Salaries\data\processed\DataScientist_y_Train.csv")

In [ ]:
# fit initial model on the training data
logistic = LogisticRegression(max_iter = 10000, random_state = 42)
logistic.fit(X_train, y_train)

#accuracy of the model on the testing data
log_score_test = logistic.score(X_test, y_test)
#predictions for the model on the testing data
log_pred_test = logistic.predict(X_test)
#Precision of the model on the testing data
log_precision_test = precision_score(y_test, log_pred_test)
#Recall of the model on the testing data
log_recall_test = recall_score(y_test, log_pred_test)

#Print a simple confusion matrix of the teting data results
print("Logistic Regression Model Testing Data Confusion Matrix :")
print(confusion_matrix(y_test, log_pred_test)) 
#Print the scores
print("Accuracy: ", log_score_test)
print("Sensitivity: ", log_recall_test)
print("Specificity: ", log_precision_test)

In [ ]:
#set a variable for the logistic intercept
log_intercept = logistic.intercept_ 
beta_0 = log_intercept

# extract log reg coefs
coef = logistic.coef_[0]
coef = np.array(coef)
df_coef = pd.DataFrame(coef)
df_coef = df_coef.T # transpose to match column names

# column names
names = X.columns
df_coef.columns = names
df_coef = df_coef.T # transpose for better view

# sort coefficients in ascending order
df_coef = df_coef.rename(columns = {0:'logregCV_coeff'})
df_coef = df_coef.sort_values('logregCV_coeff')
df_coef = df_coef.reset_index()
df_coef = df_coef.rename(columns = {'index':'Variable_Names', 'logreg_coeff':'logregCV_coeff'})
df_coef

In [ ]:
# display barplot of log reg coefs
plt.figure(figsize=(12, 8))
sns.barplot(x="Variable_Names",y="logregCV_coeff",data= df_coef)
plt.xticks(rotation=80)
plt.title('The Barplot of Coefficients of Logistic Regression Model')
plt.xlabel('Variable Names')
plt.ylabel('Value of Coefficient')
plt.show()

In [ ]:
#create a dataframe of the the True values, False values, True Predictions, and False Predicitons for a confusion matrix
cm = pd.DataFrame(confusion_matrix(y_test, log_pred_test), index=['One_Claim','Mutliple_Claims'], 
                  columns=['One_Claim','Mutliple_Claims'])
cm.index.name = 'True'
cm.columns.name = 'Predicted'

In [ ]:
# plot ROC curve
y_pred_prob = logistic.predict_proba(X_test)[:,1]

fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

plt.plot([0, 1], [0, 1],'k--')
plt.plot(fpr, tpr, label='Logistic Regression')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Decision Tree ROC Curve')
plt.show();

# calculate roc curve
fpr, tpr, thresholds = roc_curve(y_test, logistic.predict(X_test))

# roc auc score
log_roc_auc = roc_auc_score(y_test, y_pred_prob)
roc_auc_format = 'ROC AUC Score: {0:.4f}'.format(log_roc_auc)
print(roc_auc_format)

In [ ]:
df_scores = pd.DataFrame({'Measures':['Accuracy','Sensitivity','Specificity','AUC'],'Logistic':[log_score,log_recall,log_precision,log_roc_auc]})
df_scores.head()

In [ ]:
X_test = pd.read_csv(r"C:\Users\Danny Farone\Documents\GitHub\Data-Science-Job-Salaries\data\processed\Machine_Learning_Engineer_X_Test.csv")
X_train = pd.read_csv(r"C:\Users\Danny Farone\Documents\GitHub\Data-Science-Job-Salaries\data\processed\Machine_Learning_Engineer_y_Test.csv")
y_test = pd.read_csv(r"C:\Users\Danny Farone\Documents\GitHub\Data-Science-Job-Salaries\data\processed\Machine_Learning_Engineer_y_Train.csv")
y_train = pd.read_csv(r"C:\Users\Danny Farone\Documents\GitHub\Data-Science-Job-Salaries\data\processed\Machine_Learning_Engineerr_X_Train.csv")

In [ ]:
# fit initial model on the training data
logistic = LogisticRegression(max_iter = 10000, random_state = 42)
logistic.fit(X_train, y_train)

#accuracy of the model on the testing data
log_score_test = logistic.score(X_test, y_test)
#predictions for the model on the testing data
log_pred_test = logistic.predict(X_test)
#Precision of the model on the testing data
log_precision_test = precision_score(y_test, log_pred_test)
#Recall of the model on the testing data
log_recall_test = recall_score(y_test, log_pred_test)

#Print a simple confusion matrix of the teting data results
print("Logistic Regression Model Testing Data Confusion Matrix :")
print(confusion_matrix(y_test, log_pred_test)) 
#Print the scores
print("Accuracy: ", log_score_test)
print("Sensitivity: ", log_recall_test)
print("Specificity: ", log_precision_test)

In [ ]:
#set a variable for the logistic intercept
log_intercept = logistic.intercept_ 
beta_0 = log_intercept

# extract log reg coefs
coef = logistic.coef_[0]
coef = np.array(coef)
df_coef = pd.DataFrame(coef)
df_coef = df_coef.T # transpose to match column names

# column names
names = X.columns
df_coef.columns = names
df_coef = df_coef.T # transpose for better view

# sort coefficients in ascending order
df_coef = df_coef.rename(columns = {0:'logregCV_coeff'})
df_coef = df_coef.sort_values('logregCV_coeff')
df_coef = df_coef.reset_index()
df_coef = df_coef.rename(columns = {'index':'Variable_Names', 'logreg_coeff':'logregCV_coeff'})
df_coef

In [ ]:
# display barplot of log reg coefs
plt.figure(figsize=(12, 8))
sns.barplot(x="Variable_Names",y="logregCV_coeff",data= df_coef)
plt.xticks(rotation=80)
plt.title('The Barplot of Coefficients of Logistic Regression Model')
plt.xlabel('Variable Names')
plt.ylabel('Value of Coefficient')
plt.show()

In [ ]:
#create a dataframe of the the True values, False values, True Predictions, and False Predicitons for a confusion matrix
cm = pd.DataFrame(confusion_matrix(y_test, log_pred_test), index=['One_Claim','Mutliple_Claims'], 
                  columns=['One_Claim','Mutliple_Claims'])
cm.index.name = 'True'
cm.columns.name = 'Predicted'

In [ ]:
# plot ROC curve
y_pred_prob = logistic.predict_proba(X_test)[:,1]

fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

plt.plot([0, 1], [0, 1],'k--')
plt.plot(fpr, tpr, label='Logistic Regression')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Decision Tree ROC Curve')
plt.show();

# calculate roc curve
fpr, tpr, thresholds = roc_curve(y_test, logistic.predict(X_test))

# roc auc score
log_roc_auc = roc_auc_score(y_test, y_pred_prob)
roc_auc_format = 'ROC AUC Score: {0:.4f}'.format(log_roc_auc)
print(roc_auc_format)

In [ ]:
df_scores = pd.DataFrame({'Measures':['Accuracy','Sensitivity','Specificity','AUC'],'Logistic':[log_score,log_recall,log_precision,log_roc_auc]})
df_scores.head()

In [ ]:
# fit new model with best parameters determined by gscv on the training data
decTree_gs = GradientBoostingClassifier(random_state =42, learning_rate =0.01, max_depth =3, max_leaf_nodes =2, n_estimators =50)
decTree_gs.fit(X_train, y_train)

#accuracy of the model on the testing data
decTree_score_test = decTree_gs.score(X_test, y_test)
#predictions for the model on the testing data
decTree_pred_test = decTree_gs.predict(X_test)
#Precision of the model on the testing data
decTree_precision_test = precision_score(y_test, decTree_pred_test)
#Recall of the model on the testing data
decTree_recall_test = recall_score(y_test, decTree_pred_test)

#Print a simple confusion matrix of the teting data results
print("Decision Tree confusion matrix:")
print(confusion_matrix(y_test, decTree_pred_test), '\n')

#Print the scores
print("Accuracy: ", decTree_score_test)
print("Sensitivity: ", recall_test)
print("Specificity: ", precision_test)